<a href="https://colab.research.google.com/github/Poorna-A/add/blob/master/Data_Vis.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [21]:
!pip install ipywidgets
!pip install pyxlsb
!pip install jupyter-dash
!pip install dash
!pip install plotly
import pandas as pd
import ipywidgets as widgets
import plotly.express as px
import matplotlib.pyplot as plt
from dash import Dash, dcc, html, Input, Output, State


In [22]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [23]:
location = r'/content/drive/MyDrive/Poorna  Akhil Workspace/Skill Networking/June 17 Alloc.xlsb'
df = pd.read_excel(location,sheet_name = 'Sheet1')

In [24]:
df.columns

Index(['Employee ID', 'Employee Name', 'Employee Type', 'OBU',
       'OBU Description', 'Customer ID', 'Customer Name', 'Project ID',
       'Project Description', 'Program Description', 'Program Manager',
       'Contract Category', 'Project Manager ID', 'Project Manager Name',
       'Project Start Date', 'Project End Date', 'Start Date', 'End Date',
       'Country', 'Location Code', 'Location', 'Location Descr', 'JobCode',
       'JobCode Descr', 'Billable?', 'Hire Reason', 'Client Project Role',
       'Client Project Role Desc', 'Alias Name', 'Amount', 'Rate Per Role',
       'Frequency', 'Currency', 'Allocation %tage', 'Submitted By',
       'Submission Date', 'Reason for Allocation', 'Allocation Status',
       'OptimUST', 'Manager ID', 'Manager Name', 'Hire Date',
       'Termination Date', 'Employee Status', 'Business Unit', 'BU Descr',
       'Department ID', 'Department Descr', 'Comments',
       'Allocation Modified Date'],
      dtype='object')

In [25]:
# Count employees per country
country_count = df['Country'].value_counts().reset_index()
country_count.columns = ['Country', 'Employee Count']

# Calculate total allocation percentage for each employee
df['Allocation %tage'] = df['Allocation %tage'].astype(float)
total_allocation = df.groupby('Employee ID')['Allocation %tage'].sum().reset_index()
total_allocation.columns = ['Employee ID', 'Total Allocation']

# Filter employees with total allocation less than 100%
under_allocated = total_allocation[total_allocation['Total Allocation'] < 100]

# Merge with the original dataframe to get country and location info
under_allocated = pd.merge(under_allocated, df[['Employee ID', 'Country', 'Location', 'Allocation %tage']], on='Employee ID')

# Sum the allocation percentages for under-allocated employees and retain necessary info
under_allocation_status = under_allocated.groupby(['Employee ID', 'Country', 'Location'])['Allocation %tage'].sum().reset_index()
under_allocation_status.columns = ['Employee ID', 'Country', 'Location', 'Total Allocation']


In [26]:
from dash import Dash, dcc, html, Input, Output, State
import plotly.express as px

# Initialize the Dash app
app = Dash(__name__)

app.layout = html.Div([
    html.H1('Employee Allocation Dashboard'),
    html.Div(id='country-message', children='Select a country to see the location bar graph.'),
    dcc.Graph(id='country-graph'),
    html.Div(id='location-message', children='Select a location to see the under-allocated employees.'),
    dcc.Graph(id='location-graph'),
    html.Div(id='under-allocation-message', children='Employees with less than 100% allocation will be displayed here.'),
    dcc.Graph(id='under-allocation-graph'),
    dcc.Store(id='selected-country', storage_type='memory'),
    dcc.Store(id='selected-location', storage_type='memory')
])


@app.callback(
    Output('country-graph', 'figure'),
    Input('country-graph', 'clickData')
)
def update_country_graph(click_data):
    fig = px.pie(country_count, names='Country', values='Employee Count', title='Employees per Country')
    return fig

@app.callback(
    [Output('location-graph', 'figure'),
     Output('selected-country', 'data')],
    Input('country-graph', 'clickData')
)
def update_location_graph(click_data):
    if click_data:
        selected_country = click_data['points'][0]['label']
        filtered_df = df[df['Country'] == selected_country]
        location_count = filtered_df['Location'].value_counts().reset_index()
        location_count.columns = ['Location', 'Employee Count']
        fig = px.bar(location_count, x='Location', y='Employee Count', title=f'Employees in {selected_country}')
        return fig, selected_country
    return px.bar(title='Select a country to see the distribution of employees by location.'), None



In [27]:
@app.callback(
    Output('under-allocation-graph', 'figure'),
    [Input('location-graph', 'clickData'),
     State('selected-country', 'data')]
)
def update_under_allocation_graph(click_data, selected_country):
    if click_data and selected_country:
        selected_location = click_data['points'][0]['label']
        print(f"Selected Location: {selected_location}")

        # Filter the dataset for the selected country and location
        filtered_df = df[(df['Country'] == selected_country) & (df['Location'] == selected_location)]

        # Group by Employee ID to get their total allocation
        grouped_df = filtered_df.groupby('Employee ID')['Allocation %tage'].sum().reset_index()

        # Define allocation percentage ranges
        bins = [0, 25, 50, 75, 100]
        labels = ['0-25%', '26-50%', '51-75%', '76-100%']

        # Bin the allocation percentages
        grouped_df['Allocation Range'] = pd.cut(grouped_df['Allocation %tage'], bins=bins, labels=labels, include_lowest=True)

        # Count employees in each allocation range
        allocation_count = grouped_df['Allocation Range'].value_counts().sort_index().reset_index()
        allocation_count.columns = ['Allocation Range', 'Employee Count']

        print(f"Allocation Count DataFrame:\n{allocation_count}")

        # Create the bar chart
        fig = px.bar(allocation_count, x='Allocation Range', y='Employee Count',
                     title=f'Employees by Allocation Percentage in {selected_location}')
        return fig
    return px.bar(title='Select a location to see employees by allocation percentage.')


In [28]:
under_allocation_status

,Employee ID,Country,Location,Total Allocation
0,100840,India,IND-TRV,20.0
1,102113,India,IND-TRV,25.0
2,141696,India,IND-TRV,80.0
3,142196,Malaysia,MYS-Penang,85.0
4,143666,USA,USA-Working from Home,90.0
...,...,...,...,...
444,U92017,Canada,CAN-Working from Home,50.0
445,U93091,Spain,ESP-Madrid,25.0
446,U93098,USA,USA-Working from Home,60.0
447,U93790,India,IND-BLR,95.0


In [29]:
app.run_server(mode='inline')

<IPython.core.display.Javascript object>